In [ ]:
import pandas as pd
import os

csv_dir = r"C:\Users\OSHITH\OneDrive - University of Kelaniya\Desktop\lirneasia\EC_2\smart_data"
csv_files = [os.path.join(csv_dir, f"file{num}.csv") for num in range(5, 101, 5)]
output_file = os.path.join(csv_dir, "combinedSmartData.csv")

combined_df = pd.DataFrame()

for file in csv_files:
    try:
        print(f"Processing file: {file}")
        df = pd.read_csv(file)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    except FileNotFoundError:
        print(f"File not found: {file}. Skipping...")
    except Exception as e:
        print(f"Error processing {file}: {e}")
        
missing_values = combined_df.isnull().sum()
print(missing_values)

dfCleaned = combined_df.dropna()

In [ ]:
df = dfCleaned

df['DATE'] = pd.to_datetime(df['DATE'])
df['YearMonth'] = df['DATE'].dt.to_period('M')

dfMonthly = df.groupby(['household_ID', 'YearMonth']).agg({
    'TOTAL_IMPORT (kWh)': 'max',
    'TOTAL_EXPORT (kWh)': 'max',
    'TOTAL_IMPORT - PV1 (kWh)': 'max',
    'TOTAL_EXPORT - PV1 (kWh)': 'max',
    'TR1_TOTAL_IMPORT (kWh)': 'max',
    'TR2_TOTAL_IMPORT (kWh)': 'max',
    'TR3_TOTAL_IMPORT (kWh)': 'max',
    'TR1_TOTAL_EXPORT (kWh)': 'max',
    'TR2_TOTAL_EXPORT (kWh)': 'max',
    'TR3_TOTAL_EXPORT (kWh)': 'max',
    'PHASE_A_CURRENT (A)': 'mean',
    'PHASE_A_VOLTAGE (V)': 'mean',
    'FREQUENCY (Hz)': 'mean',
}).reset_index()

cumulative_columns = [
    'TOTAL_IMPORT (kWh)', 'TOTAL_EXPORT (kWh)', 'TOTAL_IMPORT - PV1 (kWh)',
    'TOTAL_EXPORT - PV1 (kWh)', 'TR1_TOTAL_IMPORT (kWh)', 'TR2_TOTAL_IMPORT (kWh)',
    'TR3_TOTAL_IMPORT (kWh)', 'TR1_TOTAL_EXPORT (kWh)', 'TR2_TOTAL_EXPORT (kWh)', 'TR3_TOTAL_EXPORT (kWh)'
]
for col in cumulative_columns:
    dfMonthly[col + '_exact'] = dfMonthly.groupby('household_ID')[col].diff().fillna(dfMonthly[col])

dfMonthly = dfMonthly.drop(columns=cumulative_columns)
print(dfMonthly.head())

In [ ]:
df = dfCleaned
householdInfo = pd.read_csv(r"C:\Users\OSHITH\OneDrive - University of Kelaniya\Desktop\lirneasia\EC_1\w1_household_information_and_history.csv")

df['DATE'] = pd.to_datetime(df['DATE'])
df['YearMonth'] = df['DATE'].dt.to_period('M')

dfMonthly = df.groupby(['household_ID', 'YearMonth']).agg({
    'TOTAL_IMPORT (kWh)': 'max'
}).reset_index()

dfMonthly['TOTAL_IMPORT (kWh)'] = dfMonthly.groupby('household_ID')['TOTAL_IMPORT (kWh)'].diff().fillna(dfMonthly['TOTAL_IMPORT (kWh)'])

dfAvgImport = dfMonthly.groupby('household_ID')['TOTAL_IMPORT (kWh)'].mean().reset_index()

dfCombined = pd.merge(dfAvgImport, householdInfo, on='household_ID', how='inner')

dfCombined.to_csv(r"C:\Users\OSHITH\OneDrive - University of Kelaniya\Desktop\lirneasia\houseConsumption.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(r"C:\Users\OSHITH\OneDrive - University of Kelaniya\Desktop\lirneasia\monthlySmartData.csv")

df['YearMonth'] = pd.to_datetime(df['YearMonth'], format='%Y-%m')

monthlySummary = df.groupby('YearMonth')[['TOTAL_IMPORT (kWh)_exact', 'TOTAL_EXPORT (kWh)_exact']].sum()

plt.figure(figsize=(10, 6))
plt.plot(monthlySummary.index, monthlySummary['TOTAL_IMPORT (kWh)_exact'], label='Total Import (kWh)', color='#FF8000', marker='o')
plt.plot(monthlySummary.index, monthlySummary['TOTAL_EXPORT (kWh)_exact'], label='Total Export (kWh)', color='#4C1F7A', marker='o')

plt.title('Total Import and Export (kWh) Over Time (Smart-Meter)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Energy (kWh)', fontsize=12)
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(r"C:\Users\OSHITH\OneDrive - University of Kelaniya\Desktop\lirneasia\monthlySmartData.csv")
df['YearMonth'] = pd.to_datetime(df['YearMonth'], format='%Y-%m')

df = df.rename(columns={
    'TR1_TOTAL_IMPORT (kWh)_exact': 'Day Time Energy (kWh)',
    'TR2_TOTAL_IMPORT (kWh)_exact': 'Peak Time Energy (kWh)',
    'TR3_TOTAL_IMPORT (kWh)_exact': 'Off-Peak Time Energy (kWh)'
})

columnsToPlot = ['Day Time Energy (kWh)', 'Peak Time Energy (kWh)', 'Off-Peak Time Energy (kWh)']
monthlySummary = df.groupby('YearMonth')[columnsToPlot].sum()

customColors = ['#FF8000', '#4C1F7A', '#219B9D']

ax = monthlySummary.plot(kind='area', stacked=True, figsize=(10, 6), color=customColors)
plt.title('Cumulative Energy Consumption per Transformer Over Time (Smart-Meter)')
plt.xlabel('Month')
plt.ylabel('Energy (kWh)')
plt.xticks(rotation=45)
plt.tight_layout()

ax.legend(title='Time Periods')

plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from sklearn.impute import SimpleImputer

df = pd.read_csv(r"C:\Users\OSHITH\OneDrive - University of Kelaniya\Desktop\lirneasia\houseConsumption.csv")

numeric_cols = df.select_dtypes(include=[np.number]).columns
df_numeric = df[numeric_cols]

imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_numeric), columns=numeric_cols)

df[numeric_cols] = df_imputed

features = df[['TOTAL_IMPORT (kWh)', 'floor_area', 'total_monthly_expenditure_of_last_month', 'no_of_household_members']]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method For Optimal K')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42)
y_kmeans = kmeans.fit_predict(scaled_features)

df['Cluster'] = y_kmeans

sns.scatterplot(x=df['TOTAL_IMPORT (kWh)'], y=df['total_monthly_expenditure_of_last_month'], hue=df['Cluster'], palette='Set1')

plt.title('K-means Clustering')
plt.xlabel('TOTAL_IMPORT (kWh)')
plt.ylabel('Total Monthly Expenditure of Last Month')
plt.show()

centroids = pd.DataFrame(kmeans.cluster_centers_, columns=['TOTAL_IMPORT (kWh)', 'floor_area', 'total_monthly_expenditure_of_last_month', 'no_of_household_members'])
print("Centroids of clusters:\n", centroids)

print("Cluster distribution:\n", df['Cluster'].value_counts())

sil_score = silhouette_score(scaled_features, y_kmeans)
print(f"Silhouette Score: {sil_score}")
